In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
def citus_row(start,end):
    def get_data_in_citus(start,end):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_citus_db",
                user="citus_bdp_usage_qa",
                host="10.2.10.132",
                password="dNzWtSV3pKTx",
                port=5432
            )
        )
        count_sql = "SELECT category_id, CAST(SUM(est_market_size_download)AS bigint)AS sum_est_market_size_download,CAST(SUM(est_market_size_revenue) AS bigint)AS sum_est_market_size_revenue, count(1) FROM store.store_market_size_fact_v1  WHERE date >='{}' AND date<='{}' AND category_id=400027  group by category_id order by category_id desc".format(start,end)
        db_data = query(citus_dsn_, count_sql)
        sum_sql = "SELECT CAST(SUM(est_market_size_download)AS bigint)AS sum_est_market_size_download,CAST(SUM(est_market_size_revenue) AS bigint)AS sum_est_market_size_revenue, count(1) FROM store.store_market_size_fact_v1  WHERE date >='{}' AND date<='{}' ".format(start,end)
        sum_data = query(citus_dsn_, sum_sql)
        print(sum_data)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(start,end)
    return [Row(category_id=r[0],sum_est_market_size_download=r[1], sum_est_market_size_revenue=r[2], count=r[3]) for r in result]
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("category_id", StringType(), True),
    StructField("sum_est_market_size_download", LongType(), True),
    StructField("sum_est_market_size_revenue", LongType(), True),
    StructField("count", LongType(), True)
    ])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data")
print("================online=======================")
online_df = generate_citus_result(spark,citus_row("2020-08-01","2020-08-01"))
online_df.show()
print("================BDL=========================")
sample_data_df=spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/")
sample_data_df.createOrReplaceTempView("sample_data")
print("Not distinct data")
spark.sql("SELECT SUM(est_market_size_download) as sum_est_market_size_download, SUM(est_market_size_revenue) as sum_est_market_size_revenue,count(1) FROM sample_data where date = '2020-08-01'  ").show()
test_sample_data_df =spark.sql("select category_id,SUM(est_market_size_download) as sum_est_market_size_download, SUM(est_market_size_revenue) as sum_est_market_size_revenue,count(1) FROM sample_data where date = '2020-08-01' and category_id=400027 and (device_code = 'android-all' or device_code = 'ios-all' or device_code = 'ios-tablet' or device_code = 'ios-phone') group by category_id order by category_id desc")
test_sample_data_df.createOrReplaceTempView("test_sample_data")
test_sample_data_df.show()

print("except all test")
test_sample_data_df.exceptAll(online_df).show()


print("Distinct data")
spark.sql("select distinct app_price_type_id, category_id, country_code, device_code, purchase_type_id, granularity, date, est_market_size_download, est_market_size_revenue FROM sample_data where date = '2020-08-01' ").createOrReplaceTempView("distinct_sample_data")

spark.sql("SELECT SUM(est_market_size_download) as sum_est_market_size_download, SUM(est_market_size_revenue) as sum_est_market_size_revenue,count(1) FROM distinct_sample_data ").show()

distinct_data_df = spark.sql("select category_id,SUM(est_market_size_download) as sum_est_market_size_download, SUM(est_market_size_revenue) as sum_est_market_size_revenue,count(1) FROM distinct_sample_data where category_id=400027  group by category_id order by category_id desc")
distinct_data_df.show()

print("except all test")
distinct_data_df.exceptAll(online_df).show()

In [0]:

spark.sql("select * from sample_data where category_id = 400027 and est_market_size_download = 23297488 and est_market_size_revenue= 813448").show()


In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/

In [0]:
%%sh
